<a href="https://colab.research.google.com/github/Saimon0007/Data-Processing/blob/main/Downstram_Pipeline_of_Proposed_Method(Retail_Customer_churn).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.metrics import (accuracy_score, confusion_matrix, roc_auc_score, f1_score)
from sklearn.metrics import (confusion_matrix, accuracy_score, f1_score, roc_auc_score, recall_score, precision_score)
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    ExtraTreesClassifier,
    VotingClassifier,
    StackingClassifier,
    AdaBoostClassifier,
    ExtraTreesClassifier,
    BaggingClassifier
)

from sklearn.linear_model import (
    LogisticRegression,
    RidgeClassifier,
    Perceptron,
    SGDClassifier,
    PassiveAggressiveClassifier
)
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

In [ ]:
df = df = pd.read_csv('https://raw.githubusercontent.com/muajnstu/Large_Scale_Implementation_of_DSK_Chain/refs/heads/main/filtered%20data/Saimon%20and%20Roni/Clustered%20retail_customer_churn.csv')

X = df.drop(columns=['Target_Churn'])
y = df['Target_Churn']

X = df.drop(columns=['Target_Churn'])
y = df['Target_Churn']

In [ ]:
y_cat = pd.Categorical(y)
y_codes = y_cat.codes

In [ ]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
print("Class distribution after SMOTE:", pd.Series(y_resampled).value_counts())

Class distribution after SMOTE: Target_Churn
0    180
2    180
1    180
3    180
4    180
5    180
Name: count, dtype: int64


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.3, random_state=46, stratify=y_resampled
)

**Implementation**

In [ ]:
def print_metrics(y_true, y_pred, y_prob=None):
    cm = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    num_classes = cm.shape[0]

    if num_classes == 2:
        TN, FP, FN, TP = cm.ravel()
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
        gmean = np.sqrt(specificity * sensitivity)
        type1 = FP / (FP + TN) if (FP + TN) > 0 else 0
        type2 = FN / (TP + FN) if (TP + FN) > 0 else 0
        fmeasure = f1_score(y_true, y_pred, pos_label=1)
        auc = 0
        if y_prob is not None and hasattr(y_prob, "shape") and y_prob.shape[1] > 1:
            try:
                auc = roc_auc_score(y_true, y_prob[:, 1])
            except Exception:
                auc = 0
    else:
        TP = np.diag(cm)
        FP = np.sum(cm, axis=0) - TP
        FN = np.sum(cm, axis=1) - TP
        TN = np.sum(cm) - (FP + FN + TP)

        specificity = np.mean([
            TN[i] / (TN[i] + FP[i]) if (TN[i] + FP[i]) > 0 else 0 for i in range(num_classes)
        ])
        sensitivity = np.mean([
            TP[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0 for i in range(num_classes)
        ])
        gmean = np.sqrt(specificity * sensitivity)
        type1 = np.mean([
            FP[i] / (FP[i] + TN[i]) if (FP[i] + TN[i]) > 0 else 0 for i in range(num_classes)
        ])
        type2 = np.mean([
            FN[i] / (TP[i] + FN[i]) if (TP[i] + FN[i]) > 0 else 0 for i in range(num_classes)
        ])
        fmeasure = f1_score(y_true, y_pred, average='macro')
        auc = 0
        if y_prob is not None and hasattr(y_prob, "shape") and y_prob.shape[1] > 1:
            try:
                auc = roc_auc_score(y_true, y_prob, multi_class='ovr', average='macro')
            except Exception:
                auc = 0

    print(f"Accuracy      : {accuracy:.4f}")
    print(f"Sensitivity   : {sensitivity:.4f}")
    print(f"Specificity   : {specificity:.4f}")
    print(f"G-Mean        : {gmean:.4f}")
    print(f"Type I Error  : {type1:.4f}")
    print(f"Type II Error : {type2:.4f}")
    print(f"F1 Score      : {fmeasure:.4f}")
    print(f"AUROC         : {auc:.4f}")

In [ ]:
def run_model(name, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    try:
        y_prob = model.predict_proba(X_test)
    except Exception:
        y_prob = None
    print(f"\nModel: {name}")
    print_metrics(y_test, y_pred, y_prob)

In [14]:
ml_models = {
    # Original models
    "RandomForest": RandomForestClassifier(random_state=42),
    "ExtraTrees": ExtraTreesClassifier(random_state=42),
    "Bagging": BaggingClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=5000, random_state=42, solver='saga'),
    "RidgeClassifier": RidgeClassifier(random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "NaiveBayes": GaussianNB(),
    "Perceptron": Perceptron(random_state=42),
    "SGDClassifier": SGDClassifier(random_state=42),
    #"KNN": KNeighborsClassifier(n_neighbors=3),
    "PassiveAggressive": PassiveAggressiveClassifier(random_state=42),
    #"RBFSVM": SVC(kernel='rbf', probability=True, random_state=42),
    "LDA": LinearDiscriminantAnalysis(),
    "QDA": QuadraticDiscriminantAnalysis(),
    "LightGBM": LGBMClassifier(verbosity=-1, random_state=42),


    "VotingSoft": VotingClassifier(
        estimators=[
            ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
            ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),
            ('et', ExtraTreesClassifier(n_estimators=100, random_state=42)),
            ('lgbm', LGBMClassifier(verbosity=-1, n_estimators=100, random_state=42))
        ],
        voting='soft',
        n_jobs=-1
    ),


    "VotingHard": VotingClassifier(
        estimators=[
            ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
            ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),
            ('et', ExtraTreesClassifier(n_estimators=100, random_state=42)),
            ('lgbm', LGBMClassifier(verbosity=-1, n_estimators=100, random_state=42))
        ],
        voting='hard',
        n_jobs=-1
    ),


    "VotingWeighted": VotingClassifier(
        estimators=[
            ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
            ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),
            ('et', ExtraTreesClassifier(n_estimators=100, random_state=42)),
            ('lgbm', LGBMClassifier(verbosity=-1, n_estimators=100, random_state=42)),
            ('xgb', XGBClassifier(n_estimators=100, random_state=42, eval_metric='mlogloss', use_label_encoder=False))
        ],
        voting='soft',
        weights=[2, 3, 2, 3, 3],
        n_jobs=-1
    ),


    "Stacking": StackingClassifier(
        estimators=[
            ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
            ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),
            ('et', ExtraTreesClassifier(n_estimators=100, random_state=42)),
            ('lgbm', LGBMClassifier(verbosity=-1, n_estimators=100, random_state=42)),
            ('xgb', XGBClassifier(n_estimators=100, random_state=42, eval_metric='mlogloss', use_label_encoder=False))
        ],
        final_estimator=LogisticRegression(max_iter=5000, random_state=42, solver='saga'),
        cv=5,
        stack_method='predict_proba',
        n_jobs=-1
    ),
}

In [15]:
ml_models["LDA"] = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
ml_models["QDA"] = QuadraticDiscriminantAnalysis(reg_param=0.01)

for name, model in ml_models.items():
    run_model(name, model, X_train, X_test, y_train, y_test)


Model: RandomForest
Accuracy      : 0.5463
Sensitivity   : 0.5463
Specificity   : 0.9093
G-Mean        : 0.7048
Type I Error  : 0.0907
Type II Error : 0.4537
F1 Score      : 0.5450
AUROC         : 0.9106

Model: ExtraTrees
Accuracy      : 0.5185
Sensitivity   : 0.5185
Specificity   : 0.9037
G-Mean        : 0.6845
Type I Error  : 0.0963
Type II Error : 0.4815
F1 Score      : 0.5183
AUROC         : 0.8908

Model: Bagging
Accuracy      : 0.5278
Sensitivity   : 0.5278
Specificity   : 0.9056
G-Mean        : 0.6913
Type I Error  : 0.0944
Type II Error : 0.4722
F1 Score      : 0.5213
AUROC         : 0.8985

Model: GradientBoosting
Accuracy      : 0.5247
Sensitivity   : 0.5247
Specificity   : 0.9049
G-Mean        : 0.6891
Type I Error  : 0.0951
Type II Error : 0.4753
F1 Score      : 0.5210
AUROC         : 0.9110


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Model: LogisticRegression
Accuracy      : 0.4321
Sensitivity   : 0.4321
Specificity   : 0.8864
G-Mean        : 0.6189
Type I Error  : 0.1136
Type II Error : 0.5679
F1 Score      : 0.4286
AUROC         : 0.8607

Model: RidgeClassifier
Accuracy      : 0.4012
Sensitivity   : 0.4012
Specificity   : 0.8802
G-Mean        : 0.5943
Type I Error  : 0.1198
Type II Error : 0.5988
F1 Score      : 0.3746
AUROC         : 0.0000

Model: DecisionTree
Accuracy      : 0.5556
Sensitivity   : 0.5556
Specificity   : 0.9111
G-Mean        : 0.7115
Type I Error  : 0.0889
Type II Error : 0.4444
F1 Score      : 0.5523
AUROC         : 0.7333

Model: NaiveBayes
Accuracy      : 0.5216
Sensitivity   : 0.5216
Specificity   : 0.9043
G-Mean        : 0.6868
Type I Error  : 0.0957
Type II Error : 0.4784
F1 Score      : 0.5199
AUROC         : 0.9109

Model: Perceptron
Accuracy      : 0.3117
Sensitivity   : 0.3117
Specificity   : 0.8623
G-Mean        : 0.5185
Type I Error  : 0.1377
Type II Error : 0.6883
F1 Score      : 

In [16]:
import io
import contextlib
import pandas as pd

# 1) Capture printed output
buffer = io.StringIO()

with contextlib.redirect_stdout(buffer):
    for name, model in ml_models.items():
        run_model(name, model, X_train, X_test, y_train, y_test)

output_text = buffer.getvalue()   # <-- now it DEFINITELY exists

# 2) Parse output into rows
rows = []
current_model = None

for line in output_text.splitlines():
    line = line.strip()

    if line.startswith("Model:"):
        current_model = {"Model": line.split(":", 1)[1].strip()}
        rows.append(current_model)

    elif ":" in line and current_model is not None:
        key, value = line.split(":", 1)
        try:
            current_model[key.strip()] = float(value.strip())
        except ValueError:
            pass

# 3) Export to CSV
df = pd.DataFrame(rows)
df.to_csv("model_outputs.csv", index=False)

print("Saved model_outputs.csv")


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Saved model_outputs.csv
